In [1]:
from pathlib import Path
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn

In [2]:
df_train = pd.read_parquet(Path("data") / "train.parquet")

In [3]:
def _encode_dates(X):
    X = X.copy()  # modify a copy of X
    # Encode the date information from the DateOfDeparture columns
    X["year"] = X["date"].dt.year
    X["month"] = X["date"].dt.month
    X["day"] = X["date"].dt.day
    X["weekday"] = X["date"].dt.weekday
    X["hour"] = X["date"].dt.hour
    X['week'] = X["date"].dt.isocalendar().week

    # Finally we can drop the original columns from the dataframe
    return X.drop(columns=["date"])

In [4]:
def _merge_external_data(X):
    file_path = file_path = Path('data') / "external_data.csv"
    df_ext = pd.read_csv(file_path, parse_dates=["date"])

    X['date'] = pd.to_datetime(X['date']).astype('datetime64[ns]')
    df_ext['date'] = pd.to_datetime(df_ext['date']).astype('datetime64[ns]')

    X = X.copy()
    # When using merge_asof left frame need to be sorted
    X["orig_index"] = np.arange(X.shape[0])
    X = pd.merge_asof(
        X.sort_values("date"), df_ext[[
            "date", "t", 'u'
        ]].sort_values("date"), on="date"
    )
    # Sort back to the original order
    X = X.sort_values("orig_index")
    del X["orig_index"]
    return X

In [5]:
df_train = _merge_external_data(df_train)

In [6]:
X_dates_encoding = _encode_dates(df_train[["date"]])
df_train = pd.concat([df_train, X_dates_encoding], axis=1)

In [7]:
y = df_train['log_bike_count']
X = df_train.drop('log_bike_count', axis=1)

In [8]:
X_train, X_valid, y_train, y_valid = train_test_split(
    X, y, random_state=2408)

In [9]:
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import Ridge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import pandas as pd
from xgboost import XGBRegressor


categorical_encoder = OneHotEncoder(handle_unknown="ignore")
categorical_cols = ["counter_name"]
passthrough_cols = ['latitude', 't', 'u', 'month', 'weekday', 'hour', 'day']

preprocessor = ColumnTransformer(
    [
        ("cat", categorical_encoder, categorical_cols),
        ('passthrough', 'passthrough', passthrough_cols)
    ]
)

regressor = XGBRegressor(
    learning_rate=0.1, max_depth=9, n_estimators=980,
    gamma=0, min_child_weight=7, reg_alpha=0.2,
    colsample_bytree=0.85, reg_lambda=6, max_delta_step=1
)

pipe = make_pipeline(preprocessor, regressor)
pipe.fit(X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['counter_name']),
                                                 ('passthrough', 'passthrough',
                                                  ['latitude', 't', 'u',
                                                   'month', 'weekday', 'hour',
                                                   'day'])])),
                ('xgbregressor',
                 XGBRegressor(base_score=None, booster=None, callbacks=None,
                              colsample_bylevel=None, colsample_bynode=None,
                              colsample_bytr...
                              feature_types=None, gamma=0, grow_policy=None,
                              importance_type=None,
                              interaction_constraints=None, learning_rate=0.1,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=1,
                              max_depth=9, max_leaves=None, min_child_weight=7,
                              missing=nan, monotone_constraints=None,
                              multi_strategy=None, n_estimators=980,
                              n_jobs=None, num_parallel_tree=None,
                              random_state=None, ...))])

In [10]:
print(
    f"Train set, RMSE={mean_squared_error(y_train, pipe.predict(X_train), squared=False):.2f}"
)
print(
    f"Test set, RMSE={mean_squared_error(y_valid, pipe.predict(X_valid), squared=False):.2f}"
)

Train set, RMSE=0.28
Test set, RMSE=0.34


In [11]:
df_test = pd.read_parquet(Path("data") / "final_test.parquet")

In [12]:
df_test = _merge_external_data(df_test)

In [13]:
test_dates_encoding = _encode_dates(df_test[["date"]])
df_test = pd.concat([df_test, test_dates_encoding], axis=1)

In [14]:
y_pred = pipe.predict(df_test)
results = pd.DataFrame(
    dict(
        Id=np.arange(y_pred.shape[0]),
        log_bike_count=y_pred,
    )
)
results.to_csv("submission.csv", index=False)